In [17]:
# Import external libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [18]:
# Set number of bits per cell
bpc = 1

In [19]:
# Analyze reliability, bandwidth, energy
for i in range(2**bpc):
    for j in range(2**bpc):
        postread_ser = (np.loadtxt(f"data/postread_{bpc}bpc_{i}_{j}.csv", delimiter=',') != j).mean()
        dt = np.loadtxt(f"data/dt_{bpc}bpc_{i}_{j}.csv", delimiter=',')
        ivdd = np.loadtxt(f"data/vdd_power_{bpc}bpc_{i}_{j}.csv", delimiter=',')
        ivdd_dac = np.loadtxt(f"data/vdd_power_{bpc}bpc_{i}_{j}.csv", delimiter=',')
        ivsa = np.loadtxt(f"data/vsa_power_{bpc}bpc_{i}_{j}.csv", delimiter=',')
        ivddio = np.loadtxt(f"data/vddio_power_{bpc}bpc_{i}_{j}.csv", delimiter=',')
        ivddio_dac = np.loadtxt(f"data/vddio_power_{bpc}bpc_{i}_{j}.csv", delimiter=',')
        power_tot = ivdd * 0.9 + ivdd_dac * 0.9 + ivsa * 0.9 + ivddio_dac * 3.3 + ivddio * 3.3
        energy_tot = power_tot * dt
        print(f"{i}->{j} transition")
        print("Post-read CER:", postread_ser)
        print("Power total:", power_tot)
        print("Total programming time:", dt)
        print("Energy total:", energy_tot)
        print("Bandwidth (b/s):", 2**16 * 48/dt)
        print("Energy per bit:", dt/2**16/48)
        print()

0->0 transition
Post-read CER: 0.015775680541992188
Power total: 0.030053342483999997
Total programming time: 3.198171854019165
Energy total: 0.09611575405152721
Bandwidth (b/s): 983601.9274720155
Energy per bit: 1.0166714522104787e-06

0->1 transition
Post-read CER: 0.36260986328125
Power total: 0.023985239186999997
Total programming time: 0.8605618476867676
Energy total: 0.020640781751973782
Bandwidth (b/s): 3655435.1188771278
Energy per bit: 2.7356524393932585e-07

0->2 transition
Post-read CER: 1.0
Power total: 0.013173851613
Total programming time: 3.0239686965942383
Energy total: 0.03983731489128951
Bandwidth (b/s): 1040264.7367159898
Energy per bit: 9.61293759852803e-07

0->3 transition
Post-read CER: 2.6702880859375e-05
Power total: 0.051860151777
Total programming time: 0.05164313316345215
Energy total: 0.0026782207240964506
Bandwidth (b/s): 60912803.06412997
Energy per bit: 1.6416909905577388e-08

1->0 transition
Post-read CER: 0.02546215057373047
Power total: 0.0327969235739